In [1]:
import pandas as pd

In [2]:
chemtax = pd.read_excel("../../data/datasets/unmodified/joywarren2019/CHEMTAX water column.xlsx")
microscopy = pd.read_excel("../../data/datasets/unmodified/joywarren2019/Joy-Warren_et_al_WAP_Primary_Production_SuppTable2.xlsx", header=1)
joyw = pd.read_excel("../../data/datasets/unmodified/joywarren2019/NBP1409_20180228_hy1_R.xlsx")

/Users/ayushnag/miniconda3/envs/sophyvenv/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/ayushnag/miniconda3/envs/sophyvenv/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/ayushnag/miniconda3/envs/sophyvenv/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [5]:
chemtax_sophy = {"CTDDEPTH": "depth", "Sta": "station", "chla_ugL": "hplc_tot_chl_a", "Chlorophytes_frac_chla": "chemtax_chlorophytes", "Dinoflagellates_frac_chla": "chemtax_dinoflagellates", "Cryptophytes_frac_chla": "chemtax_cryptophytes", "Haptophytes1_frac_chla": "chemtax_haptophytes1", "Haptophytes2_frac_chla": "chemtax_haptophytes2", "Diatoms_frac_chla": "chemtax_diatoms"}
chemtax = chemtax.rename(columns=chemtax_sophy)

microscopy_sophy = {"Exp": "experiment", "Taxa": "taxa", "Biovolume concentration (μm3 cm-3)": "biovolume_per_L"}
microscopy = microscopy.rename(columns=microscopy_sophy)

joyw_sophy = {"STNNBR": "station", "LATITUDE": "latitude", "LONGITUDE": "longitude", "CTDDEPTH": "depth", "CTDTMP_UP": "temperature", "CTDSAL_UP": "salinity", "CTDOXY_UP": "oxygen", "CTDPAR_UP": "par", "DFE": "dfe", "clor_a_flurosesc": "chl_a_fluor", "NITRAT": "nitrate", "NITRIT": "nitrite", "PHSPHT": "phosphate", "SILCAT": "silicate", "TDIC *note call DIC": "dic"}
joyw = joyw.rename(columns=joyw_sophy)

In [7]:
joyw["source"] = "joywarren2019"
# joyw["chemtax_haptophytes"] = joyw["chemtax_haptophytes1"] + joyw["chemtax_haptophytes2"]
# group by depth and station: station1([1.7, 1.8, 2.1][9.7. 9.8, 10.2]...), station2...
joyw = joyw.groupby([joyw["depth"].pct_change().abs().gt(0.15).cumsum(), "station"]).mean(numeric_only=True)
joyw = joyw.reset_index(level=0, drop=True).reset_index(level=0).sort_values(by="depth")
joyw.sort_values(by=["station", "depth"])
chemtax = chemtax.sort_values(by="depth")
# Join main sample data with chemtax by matching station and depth
joyw = pd.merge_asof(chemtax, joyw, by="station", on="depth", direction="nearest",
                                  tolerance=2).sort_values(by="station")
# Set id field for sample data so foreign keys for microscopy can be matched
joyw["id"] = joyw.index + 1
jwmkey = pd.concat([joyw["id"], joyw["station"], joyw["depth"]], axis=1).sort_values(by="depth")

date = pd.to_datetime(joyw['DATE'].fillna(0).astype(int).astype(str), format='%Y%m%d', errors='coerce')
time = pd.to_datetime(joyw['TIME'].fillna(0).astype(int).astype(str).str.zfill(4), format='%H%M', errors='coerce').dt.time
joyw["timestamp"] = pd.to_datetime(date.astype(str) + ' ' + time.astype(str), errors='coerce')

# TODO: should this be at the dataset information level? Doesn't change per sample
joyw["hplc_present"] = 1
joyw["chemtax_present"] = 1
joyw["microscopy_present"] = 1
joyw["ifcb_present"] = 0

joyw

/var/folders/wb/v7frq16s6nnb0tkx5j8nz06r0000gn/T/ipykernel_37062/2908909921.py:4: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  joyw = joyw.groupby([joyw["depth"].pct_change().abs().gt(0.15).cumsum(), "station"]).mean(numeric_only=True)


,TYPE,depth,station,CHEMTAX bin,SAMPLENAME,FLOWCAM,Chlorophytes_abs_chla,Dinoflagellates_abs_chla,Cryptophytes_abs_chla,Haptophytes1_abs_chla,...,sTA_Friis STDEV,sTA_Friis STDEV_FLAG_W,sDIC_Brown,sDIC_Brown_FLAG_W,id,timestamp,hplc_present,chemtax_present,microscopy_present,ifcb_present
60,CTD,9.8,002_02,2,18,NaN,0.000352,4.049944e-07,0.007669,0.025359,...,-999.0,9.0,-999.0,9.0,61,2014-10-31 05:02:00,1,1,1,0
39,CTD,3.5,002_02,1,16,NaN,0.011924,5.670137e-03,0.008550,0.026715,...,-999.0,9.0,-999.0,9.0,40,2014-10-31 05:03:00,1,1,1,0
200,CTD,76.3,002_02,5,34,NaN,0.022813,1.029493e-02,0.004794,0.019460,...,-999.0,9.0,-999.0,9.0,201,2014-10-31 04:57:00,1,1,1,0
99,CTD,24.7,002_02,3,25,NaN,0.000313,1.878904e-04,0.006546,0.005069,...,-999.0,9.0,-999.0,9.0,100,2014-10-31 05:00:00,1,1,1,0
223,CTD,100.1,002_02,6,36,NaN,0.009359,3.707161e-03,0.006733,0.027330,...,-999.0,9.0,-999.0,9.0,224,2014-10-31 04:55:00,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,CTD,9.9,096_01,2,3564,NaN,0.029584,2.503137e-03,0.011948,0.117429,...,-999.0,9.0,-999.0,9.0,65,2014-11-21 03:31:00,1,1,1,0
209,CTD,99.2,096_01,6,3580,NaN,0.000000,0.000000e+00,0.000000,0.008101,...,-999.0,9.0,-999.0,9.0,210,2014-11-21 03:24:00,1,1,1,0
1,CTD,1.3,096_01,1,3563,NaN,0.037369,2.036261e-03,0.012666,0.137983,...,-999.0,9.0,-999.0,9.0,2,2014-11-21 03:32:00,1,1,1,0
131,CTD,48.8,096_01,4,3573,NaN,0.013547,0.000000e+00,0.000000,0.047260,...,-999.0,9.0,-999.0,9.0,132,2014-11-21 03:28:00,1,1,1,0


In [8]:
microscopy = microscopy.dropna()
microscopy["measurement_method"] = "flowcam"
microscopy = microscopy[microscopy["Treatment"] == "initial"]
# Convert cm^3 to L (divide the volume value by 1000)
microscopy["biovolume_per_L"] = microscopy["biovolume_per_L"].div(1000)
experiment_station = {3: "016_02", 4: "046_01", 5: "050_03", 6: "052_01", 8: "058_01", 9: "061_01", 12: "078_03", 13: "082_03", 14: "088_01", 15: "066_01"}
# remove rows we don't have station info for
microscopy = microscopy[microscopy["experiment"].isin(experiment_station.keys())]
microscopy["station"] = microscopy["experiment"].map(experiment_station)
# experiment depths given in joywarren2019 paper
experiment_depth = {3: 9.9, 4: 10.2, 5: 25.4, 6: 10.0, 8: 1.8, 9: 9.7, 12: 25.2, 13: 10.1, 14: 9.9, 15: 9.9}
microscopy["depth"] = microscopy["experiment"].map(experiment_depth)

# TODO: make aphia_id column with preassigned ids for the manually replaced taxa

# List of taxa that are too general for WorMS to find an id for
replace = ("centric", "pennate", "unknown diatom", "dinoflagellate", "ciliate", "silicoflagellate")
# are_taxa = rows that have species name we can get taxonomy for (ex: Phaeocystis)
# are_taxa: Series = ~microscopy['taxa'].isin(replace)
# taxa: DataFrame = pd.read_csv(joywarren_worms_file, encoding='utf-8').rename(worms_sql)
# taxa.index = microscopy[are_taxa].index
# ----------------------------------
# microscopy['aphia_id'] = taxa['AphiaID']
microscopy = microscopy.sort_values(by='depth')
# Join microscopy data with matching id in sample table (by depth and station)
microscopy = pd.merge_asof(microscopy, jwmkey, by='station', on='depth', direction='nearest', tolerance=1)
microscopy = microscopy.rename({'id': 'sample_id', 'group': 'groups'}, axis="columns")
microscopy

,experiment,Treatment,Group,taxa,# images,biovolume_per_L,measurement_method,station,depth,sample_id
0,8,initial,pennate,Navicula,1.0,43.331605,flowcam,058_01,1.8,15
1,8,initial,centric,Proboscia,1.0,39.234955,flowcam,058_01,1.8,15
2,8,initial,centric,Thalassiosira,4.0,3187.660269,flowcam,058_01,1.8,15
3,8,initial,dinoflagellate,dinoflagellate,3.0,0.455327,flowcam,058_01,1.8,15
4,8,initial,pennate,Banquisia,5.0,96.678097,flowcam,058_01,1.8,15
...,...,...,...,...,...,...,...,...,...,...
136,5,initial,pennate,Pseudo-nitzschia,2.0,165.508378,flowcam,050_03,25.4,123
137,5,initial,Phaeocystis,Phaeocystis antarctica,354.0,263589.831657,flowcam,050_03,25.4,123
138,5,initial,silicoflagellate,silicoflagellate,3.0,0.303715,flowcam,050_03,25.4,123
139,5,initial,unknown diatom,unknown diatom,14.0,0.847877,flowcam,050_03,25.4,123


In [ ]:
joyw.to_csv("../../data/datasets/modified/joywarren2019/sample.csv", index=False)
microscopy.to_csv("../../data/datasets/modified/joywarren2019/sample_amount.csv", index=False)

In [ ]:
joyw["id"] = joyw.index + 1
# make a local id
# join the sample and microscopy tables (using whatever depth and station)
# - when joined, the id's will be a column that is added to the joined result